In [5]:
!pip install beautifulsoup4
!pip install requests
!pip install unidecode


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/235.5 kB ? eta -:--:--
     -----------------------------------   225.3/235.5 kB 13.4 MB/s eta 0:00:01
     -------------------------------------- 235.5/235.5 kB 4.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
sports_dict = {"Archery": [" (archer)"],
               "Athletics": [" (athlete)"],
               "Badminton": [" (badminton)"],
               "Baseball": [" (baseball)"],
               "Basketball": [" (basketball)"],
               "Boxing": [" (boxer)"],
               "Canoeing": [" (canoeist)", " (kayaker)"],
               "Cycling": [" (cyclist)"],
               "Diving": [" (diver)"],
               "Equestrian": [" (equestrian)"],
               "Fencing": [" (fencer)"],
               "Field hockey": [" (field hockey)"],
               "Football": [" (footballer)"],
              "Golf": [" (golfer)"],
               "Gymnastics": [" (gymnast)"],
               "Handball": [" (handballer)"],
              "Judo": [" (judoka)"],
               "Karate": [" (karateka)"],
                "Modern pentathlon": [" (pentathlete)"],
               "Rowing": [" (rower)"],
               "Rugby sevens": [" (rugby union)"],
               "Sailing": [" (sailor)"],
               "Shooting": [" (sport shooter)"],
               "Skateboarding": [" (skateboarder)"],
               "Softball": [" (softball)"],
               "Sport Climbing": [" (climber)"],
               "Surfing": [" (surfer)"],
               "Synchronized swimming": [" (synchronized swimmer)"],
               "Swimming": [" (swimmer)"],
               "Table tennis": [" (table tennis)"],
               "Tennis": [" (tennis)"],
               "Taekwondo": [" (taekwondo)"],
               "Triathlon": [" (triathlete)"],
              "Volleyball": [" (voleyball)", " (beach voleyball)"],
              "Water polo": [" (water polo)"],
              "Weightlifting": [" (weightligter)"],
               "Wrestling": [" (wrestler)"]
              }

In [93]:
import pandas as pd

# Create an empty DataFrame
df = pd.DataFrame(columns=["Name", "Medal", "Sport", "Event", "Date", "Team"])

# Print the empty DataFrame
print(df)

Empty DataFrame
Columns: [Name, Medal, Sport, Event, Date, Team]
Index: []


In [94]:
years =  [*range(1992, 2000, 4)] 
years =  [*range(1972, 2023, 4)] 
years

[1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]

In [95]:
def Athlete_Place_FullName(soup):
    row = soup.find('li', class_='interlanguage-link interwiki-es mw-list-item')
    if row:
        link = row.find('a', class_='interlanguage-link-target')
        if link:
            spain_wikipedia_url = link['href']
            print("Link to the Spanish Wikipedia page:", spain_wikipedia_url)
            response = requests.get(spain_wikipedia_url)
            
            spa_soup = BeautifulSoup(response.content, "html.parser")
            try:
                born_place = spa_soup.find(lambda tag: tag.name == "a" and tag.has_attr("href") and "Nacimiento" in tag.find_all_previous(string=True)).text
            except:
                born_place = 'None'
            try:
                full_name = spa_soup.find('p').find('b').text
            except:
                full_name = 'None'
                
            return born_place, full_name
        else:
            print("Link not found in the row.")
    else:
        print("Row not found.")

In [96]:
def Athlete_Scrapper(name, sport, year, sports_dict):
    article = False
    attempts = 0
    url_medalist = "https://en.wikipedia.org/wiki/"+str(name)
    while (article == False):
        response = requests.get(url_medalist, verify=False)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.content, "html.parser")
            text_article = soup.get_text()
            if (sport[:5].lower() in text_article) and (str(year) in text_article):
                # English article found
                article = True
                # We get the birth
                birth = soup.find("span", {"class": "bday"})
                if birth == None:
                    birth = re.search(r'\d{1,2} [A-Z][a-z]+ \d{4}', text_article)
                    
                # For the born-place and Full-name we go to the Spanish wikipedia
                place, full_name = Athlete_Place_FullName(soup)
                
                #name = soup.find("td", {"class": "infobox-data nickname"}).text.strip()  # word in mayus after Surname
                #birth =  soup.find("span", {"class": "birthplace"}).text.strip()
                #splitted_birth = birth.split(',')[0]
                #print(name, splitted_birth, "aaaaaaa")
                print(birth, "aaaaaaa")
                article = True
                return birth, place, full_name
            else:
                print("Wrong Article")
                article = False
                attempts += 1
        else:
            print("Article not found")
            article = False
            attempts += 1
        if attempts > 2:
            article = True
            print("Not found :(")
        if article == False:    
            print("Trying another link...")
            try:
                url_medalist = "https://en.wikipedia.org/wiki/"+str(name)+sports_dict[sport][attempts-1]
            except:
                article = True
                print("Not found :(")
                return 'None', 'None', 'None'

In [107]:
import warnings
warnings.filterwarnings("ignore")

import requests
import re
from bs4 import BeautifulSoup
from unidecode import unidecode


for year in years:
    print("Year: ", year)
    # Define the URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/Spain_at_the_"+str(year)+"_Summer_Olympics"

    # Send an HTTP GET request to the URL
    response = requests.get(url, verify=False)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, "html.parser")
    
        # Find the "Medalists" section by searching for the corresponding header
        #print(soup)
        medalists_section = soup.find("span", {"id": "Medalists"})
    
        if medalists_section:
            # Find the table that follows the "Medalists" header
            medalists_table = medalists_section.find_next("table")
    
            # Extract and print the table rows and data
            for row in medalists_table.find_all("tr"):
                cells = row.find_all(["th", "td"])
                row_data = [cell.get_text(strip=True) for cell in cells]
                if row_data[0] in ["Gold", "Silver", "Bronze"]: # The row is a medal
                    words = row_data[1].split()
                    names = [row_data[1]]
                    pattern = r'[a-z][A-Z]'
                    #pattern = r'[a-zà-úÁ-ÖØ-öø-ú]
                    if re.search(pattern, unidecode(row_data[1])):
                        string = row_data[1]
                        #names = re.findall(r'[A-Z][a-z]+(?:[- ][A-Z][a-z]+)+', string)
                        #names = re.findall(r'[A-ZÀ-Ú][a-zà-ú]+(?:[- ][A-ZÀ-Ú][a-zà-ú]+)+', string)
                        names = re.findall(r'[A-ZÀ-Úa-zA-ZćØ][a-zà-úćØ]+(?:[- ][A-ZÀ-Úa-zA-ZćØ][a-zà-úćØ]+)+', string)
                        names = [name for name in names if sum(1 for letter in name if letter.isupper()) >= 2]
                    print("Team:", names)
                    for name in names:
                        # We extract the personal information
                        try:
                            date = row_data[4]
                        except:
                            date = "-"
                        
                        print(name, row_data[2], year)
                        birth = None
                        place = None
                        full_name = None
                        #birth, place, full_name = Athlete_Scrapper(name, row_data[2], year, sports_dict)
                        new_row = {"Name": name, "Full Name": full_name, "Birth": birth, "Born place": place, "Medal": row_data[0], "Sport": row_data[2], "Event": row_data[3], "Date": date, "Team": len(names), "Year": year}
                        # Use the append() method to add the new row
                        df = df.append(new_row, ignore_index=True)
                        
                        
    
        else:
            print("Medalists section not found on the page.")
    else:
        print("Failed to retrieve the page. Status code:", response.status_code)

Year:  1972
bbb <tr>
<td><span data-sort-value="03 !"><span class="mw-default-size" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="16" data-file-width="16" decoding="async" height="16" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/89/Bronze_medal_icon.svg/16px-Bronze_medal_icon.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/89/Bronze_medal_icon.svg/24px-Bronze_medal_icon.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/89/Bronze_medal_icon.svg/32px-Bronze_medal_icon.svg.png 2x" width="16"/></span></span> Bronze</span></td>
<td><span data-sort-value="Rodríguez, Enrique"><span class="vcard"><span class="fn"><a href="/wiki/Enrique_Rodr%C3%ADguez_(boxer)" title="Enrique Rodríguez (boxer)">Enrique Rodríguez</a></span></span></span></td>
<td><a href="/wiki/Boxing_at_the_1972_Summer_Olympics" title="Boxing at the 1972 Summer Olympics">Boxing</a></td>
<td><a href="/wiki/Boxing_at_the_1972_Summer_Olympics" title="Boxing at t

NameError: name 'aaaa' is not defined

In [92]:
import re

data = ["Spain men's national basketball teamNikola MirotićNikola Mirotić-Savić de-uauFernando RomayJuan Antonio San EpifanioIgnacio SolozábalJuan Manuel López IturriagaJosé MargallFernando Martín EspinaJuan de la CruzAndrés JiménezJosé LlorenteFernando ArcegaJosé Manuel BeiránJuan Corbalán"]

# Use regular expressions to find player names
player_names = re.findall(r'[A-Z][a-zÀ-ÖØ-öø-ú]+(?: [A-Z][a-zÀ-ÖØ-öø-ú]+)+', data[0])
player_names = re.findall(r'[A-Z][a-zÀ-ÖØ-öø-ú]+(?:[- ][A-Z][a-zÀ-ÖØ-öø-ú]+)+', data[0])
player_names = re.findall(r'[A-Z][a-z]+(?: [A-Z][a-z]+)+', data[0])
player_names = re.findall(r'[A-ZÀ-Ú][a-zà-ú]+(?:[- ][A-ZÀ-Ú][a-zà-ú]+)+', data[0])
player_names = re.findall(r'[A-ZÀ-Úa-zA-ZćØ][a-zà-úćØ]+(?:[- ][A-ZÀ-Ú][a-zà-úćØ]+)+', data[0])
player_names = re.findall(r'[A-ZÀ-Úa-zA-ZćØ][a-zà-úćØ]+(?:[- ][A-ZÀ-Úa-zA-ZćØ][a-zà-úćØ]+)+', data[0]) # daleee

filtered_names = [name for name in player_names if sum(1 for letter in name if letter.isupper()) >= 2]
filtered_names



#NO L'AGAFA PK d i l SON MINUSCULA de la

['Nikola Mirotić',
 'Nikola Mirotić-Savić de-uau',
 'Fernando Romay',
 'Juan Antonio San Epifanio',
 'Ignacio Solozábal',
 'Juan Manuel López Iturriaga',
 'José Margall',
 'Fernando Martín Espina',
 'Juan de la Cruz',
 'Andrés Jiménez',
 'José Llorente',
 'Fernando Arcega',
 'José Manuel Beirán',
 'Juan Corbalán']

In [99]:
df
# 517 rows
df.to_csv('mi_archivo.csv', index=False)  # El parámetro index=False evita que se escriba el índice en el archivo CSV

In [98]:
df

,Name,Medal,Sport,Event,Date,Team,Birth,Born place,Full Name,Year
0,Enrique Rodríguez,Bronze,Boxing,Light flyweight,10 September,1,None,None,None,1972.0
1,Antonio Gorostegui,Silver,Sailing,470,July 27,2,None,None,None,1976.0
2,Pedro Luís Millet,Silver,Sailing,470,July 27,2,None,None,None,1976.0
3,José María Esteban,Silver,Canoeing,Men's K-4 1000 m,July 31,4,None,None,None,1976.0
4,José Ramón López,Silver,Canoeing,Men's K-4 1000 m,July 31,4,None,None,None,1976.0
...,...,...,...,...,...,...,...,...,...,...
512,Viran Morros,Bronze,Handball,Men's tournament,7 August,16,None,None,None,2020.0
513,Gonzalo Pérez de Vargas,Bronze,Handball,Men's tournament,7 August,16,None,None,None,2020.0
514,Miguel Sánchez-Migallón,Bronze,Handball,Men's tournament,7 August,16,None,None,None,2020.0
515,Daniel Sarmiento,Bronze,Handball,Men's tournament,7 August,16,None,None,None,2020.0


In [86]:
filtered_names

['Julen Aguinagalde',
 'Rodrigo Corrales',
 'Alex Dujshebaev',
 'Raúl Entrerríos',
 'Ángel Fernández',
 'Adrià Figueras',
 'Antonio García Robledo',
 'Aleix Gómez',
 'Gedeón Guardiola',
 'Eduardo Gurbindo',
 'Jorge Maqueda',
 'Viran Morros',
 'Gonzalo Pérez de Vargas',
 'Miguel Sánchez-Migallón',
 'Daniel Sarmiento',
 'Ferran Solé']

In [39]:
spain_wikipedia_url = 'https://ca.wikipedia.org/wiki/Mireia_Belmonte_i_Garc%C3%ADa'

response = requests.get(spain_wikipedia_url)

soup = BeautifulSoup(response.content, "html.parser")


In [40]:
from bs4 import BeautifulSoup

# Assuming you have a BeautifulSoup object named 'soup' containing the parsed HTML content.
# Replace 'soup' with your actual BeautifulSoup object.

# Define a custom function to find the first <a> element that has an 'href' attribute
def find_first_a_href_after_nacimiento(tag):
    return tag.name == "a" and tag.has_attr("href") and "Naixement" in tag.find_all_previous(string=True)

# Find the first <a href> element that appears after "Nacimiento"
first_a_href_after_nacimiento = soup.find(find_first_a_href_after_nacimiento)

# Extract the content of the first <a href> element after "Nacimiento"
if first_a_href_after_nacimiento:
    print("Content of the first <a href> element after 'Naixement':", first_a_href_after_nacimiento)
else:
    print("No <a href> element found after 'Naixement'.")

Content of the first <a href> element after 'Naixement': <a href="https://www.wikidata.org/wiki/Q232218?uselang=ca#P1477" title="Modifica el valor a Wikidata"><img alt="Modifica el valor a Wikidata" class="mw-file-element" data-file-height="600" data-file-width="600" decoding="async" height="10" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/63/Arbcom_ru_editing.svg/10px-Arbcom_ru_editing.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/6/63/Arbcom_ru_editing.svg/15px-Arbcom_ru_editing.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/6/63/Arbcom_ru_editing.svg/20px-Arbcom_ru_editing.svg.png 2x" width="10"/></a>


In [46]:
rows_with_href = soup.find_all(lambda tag: tag.name == "a" and tag.has_attr("href") and "anys" in tag.find_all_previous(string=True))
print(rows_with_href)

# Find the first row with a "title" attribute
aa = rows_with_href[0].text
aa

[]


IndexError: list index out of range

In [21]:
first_p_b = soup.find('p').find('b')

# Extract the content of the first <p><b> element
if first_p_b:
    first_p_b_content = first_p_b.text
    print("Content of the first <p><b> element:", first_p_b_content)
else:
    print("No <p><b> element found.")

Content of the first <p><b> element: Paula Klamburg i Roque


In [72]:
born_place = soup.find(lambda tag: tag.name == "a" and tag.has_attr("href") and "Nacimiento" in tag.find_all_previous(string=True)).text
full_name = soup.find('p').find('b').text
print(born_place, full_name)

Tegueste Pedro González López


In [23]:
name = "Ricardo Cabot Durán"

# Use regular expressions to remove the content within angle brackets
name_cleaned = re.sub(r'«.*?»', '', name).replace("  ", " ")
final = name_cleaned.replace("  ", " ")
final

'Ricardo Cabot Durán'

In [22]:
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="ca">
<head>
<meta charset="utf-8"/>
<title>Paula Klamburg i Roque - Viquipèdia, l'enciclopèdia lliure</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled v

In [41]:
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>«»J«»u«»a«»n«» «»L«»u«»i«»s«» «»C«»o«»g«»h«»e«»n«» «»-«» «»W«»i«»k«»i«»p«»e«»d«»i«»a«»,«» «»l«»a«» «»e«»n«»c«»i«»c«»l«»o«»p«»e«»d«»i«»a«» «»l«»i«»b«»r«»e«»</title>
<script>«»(«»f«»u«»n«»c«»t«»i«»o«»n«»(«»)«»{«»v«»a«»r«» «»c«»l«»a«»s«»s«»N«»a«»m«»e«»=«»"«»c«»l«»i«»e«»n«»t«»-«»j«»s«» «»v«»e«»c«»t«»o«»r«»-«»f«»e«»a«»t«»u«»r«»e«»-«»l«»a«»n«»g«»u«»a«»g«»e«»-«»i«»n«»-«»h«

In [47]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [48]:
from textblob import TextBlob

def traducir_a_catalan(nombre_poblacion):
    try:
        blob = TextBlob(nombre_poblacion)
        translation = blob.translate(to='ca')
        return str(translation)
    except Exception as e:
        return nombre_poblacion

# Ejemplos de nombres de poblaciones en castellano
nombres_castellano = ["Moncada y Reixach", "Tarrasa", "Gerona"]

# Traducir a catalán
nombres_catalan = [traducir_a_catalan(nombre) for nombre in nombres_castellano]

# Resultados
for nombre_castellano, nombre_catalan in zip(nombres_castellano, nombres_catalan):
    print(f"{nombre_castellano} --> {nombre_catalan}")


Moncada y Reixach --> Moncada y Reixach
Tarrasa --> Tarrasa
Gerona --> Gerona


In [ ]:
# lo dels idiomes, passo